In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import datetime as dt
from scipy.spatial.distance import cdist
from sklearn.cluster import DBSCAN
from sklearn.neighbors import KDTree
from sklearn.preprocessing import RobustScaler

In [3]:
plots = pd.concat((pd.read_csv(r'C:\Users\Yanir Harel\Documents\python-stuff\clustering\blue_tracks_absolute.csv'), pd.read_csv(r'C:\Users\Yanir Harel\Documents\python-stuff\clustering\red_tracks_absolute.csv')), axis=0)

In [4]:
plots = plots.drop(['Unnamed: 0', 'BlueTrackId', 'RedTrackId'], axis=1)
plots['epoch'] = (pd.to_datetime(plots['TimeStamp']) - dt.datetime(1970, 1, 1)).dt.total_seconds()

In [5]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()

In [6]:
spatial_plots = RobustScaler().fit_transform(plots[['Lat', 'Lon', 'Alt', 'epoch']])

kdt = KDTree(spatial_plots, metric='euclidean')

k = 2 * spatial_plots.shape[-1] - 1

dist, ind = kdt.query(spatial_plots, k=k)

dist = np.sort(dist, axis=0)
dist = dist[:, k-1]
px.line(dist)

In [7]:
model = DBSCAN(eps=0.1581433, min_samples=spatial_plots.shape[1] * 2)
plots['clusterid'] = model.fit_predict(spatial_plots).astype('str')

In [10]:
plots['clusterid'] = [1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
-1,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
3,
3,
3,
3,
3,
3,
3,
3,
3,
4,
4,
4,
4,
4,
4,
1,
1,
1,
1,
1,
1,
1]

In [11]:
fig = px.scatter_3d(plots, x='Lon', y='Lat', z='Alt', color='clusterid', hover_name='TimeStamp', template='plotly_dark', labels=None)
fig = fig.update_traces(marker={'size': 2}).update_scenes(xaxis_showticklabels=False, yaxis_showticklabels=False, zaxis_showticklabels=False)
fig.show()